# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 24 2023 10:16AM,258121,16,9200290,Sartorius Bioprocess Solutions,Released
1,Feb 24 2023 10:14AM,258120,19,60020317,"GUSA Granules USA, Inc.",Released
2,Feb 24 2023 9:51AM,258118,10,9197224,Eywa Pharma Inc.,Released
3,Feb 24 2023 9:51AM,258118,10,9197225,Eywa Pharma Inc.,Released
4,Feb 24 2023 9:47AM,258119,10,CTF0011654,"Citieffe, Inc.",Released
5,Feb 24 2023 9:41AM,258117,15,9197227,"Mizner Bioscience, LLC",Released
6,Feb 24 2023 9:41AM,258117,15,9197228,"Mizner Bioscience, LLC",Released
7,Feb 24 2023 9:40AM,258075,15,VT80184429,"Virtus Pharmaceuticals, LLC",Released
8,Feb 24 2023 9:40AM,258075,15,VT80184430,"Virtus Pharmaceuticals, LLC",Released
9,Feb 24 2023 9:40AM,258075,15,VT80184431,"Virtus Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,258117,Released,2
30,258118,Released,2
31,258119,Released,1
32,258120,Released,1
33,258121,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
258117,NaN,2.0
258118,NaN,2.0
258119,NaN,1.0
258120,NaN,1.0
258121,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
258041,0.0,1.0
258044,1.0,0.0
258052,0.0,1.0
258053,3.0,7.0
258058,0.0,48.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
258041,0,1
258044,1,0
258052,0,1
258053,3,7
258058,0,48


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,258041,0,1
1,258044,1,0
2,258052,0,1
3,258053,3,7
4,258058,0,48


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,258041,,1
1,258044,1,
2,258052,,1
3,258053,3,7
4,258058,,48


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 24 2023 10:16AM,258121,16,Sartorius Bioprocess Solutions
1,Feb 24 2023 10:14AM,258120,19,"GUSA Granules USA, Inc."
2,Feb 24 2023 9:51AM,258118,10,Eywa Pharma Inc.
4,Feb 24 2023 9:47AM,258119,10,"Citieffe, Inc."
5,Feb 24 2023 9:41AM,258117,15,"Mizner Bioscience, LLC"
7,Feb 24 2023 9:40AM,258075,15,"Virtus Pharmaceuticals, LLC"
11,Feb 24 2023 9:35AM,258074,15,"Alliance Pharma, Inc."
23,Feb 24 2023 9:13AM,258115,10,Bio-PRF
34,Feb 24 2023 8:52AM,258111,10,Emerginnova
36,Feb 24 2023 8:50AM,258110,12,Hush Hush


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Feb 24 2023 10:16AM,258121,16,Sartorius Bioprocess Solutions,,1
1,Feb 24 2023 10:14AM,258120,19,"GUSA Granules USA, Inc.",,1
2,Feb 24 2023 9:51AM,258118,10,Eywa Pharma Inc.,,2
3,Feb 24 2023 9:47AM,258119,10,"Citieffe, Inc.",,1
4,Feb 24 2023 9:41AM,258117,15,"Mizner Bioscience, LLC",,2
5,Feb 24 2023 9:40AM,258075,15,"Virtus Pharmaceuticals, LLC",,4
6,Feb 24 2023 9:35AM,258074,15,"Alliance Pharma, Inc.",,12
7,Feb 24 2023 9:13AM,258115,10,Bio-PRF,,11
8,Feb 24 2023 8:52AM,258111,10,Emerginnova,,2
9,Feb 24 2023 8:50AM,258110,12,Hush Hush,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 24 2023 10:16AM,258121,16,Sartorius Bioprocess Solutions,1,
1,Feb 24 2023 10:14AM,258120,19,"GUSA Granules USA, Inc.",1,
2,Feb 24 2023 9:51AM,258118,10,Eywa Pharma Inc.,2,
3,Feb 24 2023 9:47AM,258119,10,"Citieffe, Inc.",1,
4,Feb 24 2023 9:41AM,258117,15,"Mizner Bioscience, LLC",2,
5,Feb 24 2023 9:40AM,258075,15,"Virtus Pharmaceuticals, LLC",4,
6,Feb 24 2023 9:35AM,258074,15,"Alliance Pharma, Inc.",12,
7,Feb 24 2023 9:13AM,258115,10,Bio-PRF,11,
8,Feb 24 2023 8:52AM,258111,10,Emerginnova,2,
9,Feb 24 2023 8:50AM,258110,12,Hush Hush,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 24 2023 10:16AM,258121,16,Sartorius Bioprocess Solutions,1,
1,Feb 24 2023 10:14AM,258120,19,"GUSA Granules USA, Inc.",1,
2,Feb 24 2023 9:51AM,258118,10,Eywa Pharma Inc.,2,
3,Feb 24 2023 9:47AM,258119,10,"Citieffe, Inc.",1,
4,Feb 24 2023 9:41AM,258117,15,"Mizner Bioscience, LLC",2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Feb 24 2023 10:16AM,258121,16,Sartorius Bioprocess Solutions,1.0,NaN
1,Feb 24 2023 10:14AM,258120,19,"GUSA Granules USA, Inc.",1.0,NaN
2,Feb 24 2023 9:51AM,258118,10,Eywa Pharma Inc.,2.0,NaN
3,Feb 24 2023 9:47AM,258119,10,"Citieffe, Inc.",1.0,NaN
4,Feb 24 2023 9:41AM,258117,15,"Mizner Bioscience, LLC",2.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 24 2023 10:16AM,258121,16,Sartorius Bioprocess Solutions,1.0,0.0
1,Feb 24 2023 10:14AM,258120,19,"GUSA Granules USA, Inc.",1.0,0.0
2,Feb 24 2023 9:51AM,258118,10,Eywa Pharma Inc.,2.0,0.0
3,Feb 24 2023 9:47AM,258119,10,"Citieffe, Inc.",1.0,0.0
4,Feb 24 2023 9:41AM,258117,15,"Mizner Bioscience, LLC",2.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4645489,180.0,3.0
12,516154,1.0,1.0
15,774266,18.0,0.0
16,258121,1.0,0.0
19,1290464,21.0,37.0
20,258052,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4645489,180.0,3.0
1,12,516154,1.0,1.0
2,15,774266,18.0,0.0
3,16,258121,1.0,0.0
4,19,1290464,21.0,37.0
5,20,258052,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,180.0,3.0
1,12,1.0,1.0
2,15,18.0,0.0
3,16,1.0,0.0
4,19,21.0,37.0
5,20,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,180.0
1,12,Released,1.0
2,15,Released,18.0
3,16,Released,1.0
4,19,Released,21.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20
Status,,,,,,
Executing,3.0,1.0,0.0,0.0,37.0,0.0
Released,180.0,1.0,18.0,1.0,21.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20
0,Executing,3.0,1.0,0.0,0.0,37.0,0.0
1,Released,180.0,1.0,18.0,1.0,21.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20
0,Executing,3.0,1.0,0.0,0.0,37.0,0.0
1,Released,180.0,1.0,18.0,1.0,21.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()